In [1]:
import autograd.numpy as np
from autograd import grad
from autograd.extend import defvjp
from scipy import optimize

In [2]:
def pinv_vjp(g, A1, A):
    #A1 = np.linalg.pinv(A)
    In = np.eye(A.shape[1])
    Im = np.eye(A.shape[0])
    term1 = -np.dot(A1, np.dot(g.T, A1))
    term2 = np.dot(np.dot(A1, A1.T), np.dot(g, Im - np.dot(A, A1)))
    term3 = np.dot(In - np.dot(A1, A), np.dot(g, np.dot(A1.T, A1)))
    return (term1 + term2 + term3).T

defvjp(np.linalg.pinv, lambda A1, A: lambda g: pinv_vjp(g, A1, A))

In [3]:
"""
For an explicit workload, assume we already have noisy answers to all the queries 
(say, at variance 1), and we want to design a strategy that keeps this prior information
in mind.  For a given strategy, the estimated answer to a workload query will combine the
answer obtained by computing W A^+ y and the variance 1 estimates that are there already.  
The variance of the final answer of any workload query can be calculating using 
"inverse-variance weighting".  Importantly, gradients can be back-propagated through this
averaging procedure so we can optimize the strategy with this prior information.

This is analogous to the setting where A is a union of kronecker products, because
each workload query will be answered by at least 1 of the sub-strategies, and probably 
more than 1.  So it is important that workload error tries to take into account the 
contribution of all the sub-strategies for each query.  

Note: is there a dependence on epsilon?
"""
def per_query_error(A, W):
    AtA = np.dot(A.T, A)
    AtA1 = np.linalg.pinv(AtA) # inv vs. pinv?
    X = np.dot(W, np.dot(AtA1, W.T))
    #X = np.linalg.multi_dot([W, AtA1, W.T]) -- autograd times out when using multi_dot
    # TODO(ryan): error should be infinite if query not support by strategy
    return np.diag(X)

def supported_queries(A, W):
    AtA = np.dot(A.T, A)
    AtA1 = np.linalg.pinv(AtA) # inv vs. pinv?
    AA1 = np.dot(AtA1, AtA)
    Y = np.dot(W, AA1) - W
    return ~np.any(Y, axis=1)

def total_error(A, W):
    u = per_query_error(A, W)
    v = 1.0 / (1.0 / u + 1.0 / 1.0)
    return np.sum(v)

In [4]:
n = 8
p = 2
W = np.tril(np.ones((n,n)))
A = np.vstack([np.eye(n), np.random.rand(p,n)])
A /= np.sum(A, axis=0)

err = total_error(A, W)
derr = grad(total_error)(A,W)

In [5]:
def per_query_error_kk(A, W):
    """ error for kron strategy and kron worklaod"""
    tmp = [per_query_error(Ai, Wi)[:,None] for Ai, Wi in zip(A, W)]
    return reduce(np.kron, tmp).flatten()

def per_query_error_kron(A, W):
    """ error for kron strategy and union-kron worklaod """
    return np.concatenate([per_query_error_kk(A, Wk) for Wk in W])

def total_error_union_kron(A, W, eps = None):
    """ error upper bound for union-kron strategy and union-kron workload"""
    # assumes each Al has sensitivity 1
    # TODO(ryan): remove this assumption
    if eps is None:
        eps = np.ones(len(A))
    V = np.array([per_query_error_kron(Al, W) for Al in A]) / eps[:,None]**2
    s = np.sum(1.0 / V, axis=0)
    f = np.sum(1.0 / s)
    return f * np.sum(eps)**2

In [7]:
from experiments.census_workloads import CensusSF1

sf1 = CensusSF1()
Wsf1 = [[S.W for S in K.workloads] for K in sf1.workloads]
I = np.eye
T = lambda n: np.ones((1,n))
def pid(p, n, params):
    B = np.reshape(params, (p,n))
    A = np.vstack([I(n), B])
    return A / np.sum(A, axis=0)

def census_parameterization(params):
    S1 = pid(8, 64, params[:8*64])
    S2 = pid(8, 64, params[8*64:16*64])
    S3 = pid(10, 115, params[16*64:-2])
    eps = params[-2:]
    
    # these approximations of total are necessary because queries
    # not supported by a sub-strategy are not handled correctly
    T17 = pid(1, 17, 100*np.ones(17))
    T64 = pid(1, 64, 100*np.ones(64))
    T115 = pid(1, 115, 100*np.ones(115))
    
    A1 = [I(2), I(2), S1, T17, T115]
    A2 = [I(2), I(2), S2, T17, S3]
    return [A1, A2], eps / np.sum(eps)

def obj(params):
    A, eps = census_parameterization(params)
    ans = total_error_union_kron(A, Wsf1)
    #print ans, eps
    return ans

eye = [[I(n) for n in sf1.domain]]
eye[0][3] = T(17)
baseline = total_error_union_kron(eye, Wsf1)
print 'Identity', baseline

init = np.random.rand(16*64 + 10*115 + 2)
init[-2] = 5.0

print obj(init)
print grad(obj)(init)
bnds = [(0,None)]*len(init)
opts={}#{'maxiter':25}

res = optimize.minimize(obj, init, method='L-BFGS-B', jac=grad(obj), bounds=bnds, options=opts)

SyntaxError: invalid syntax (<ipython-input-7-afcca94a496f>, line 42)

In [133]:
import matplotlib.pyplot as plt
Asf1, _ = census_parameterization(res.x)
#A = Asf1[1][4]
#plt.plot(A[115:].T)

print np.sqrt(baseline / total_error_union_kron(Asf1, Wsf1))

e1 = per_query_error_kron(Asf1[0], Wsf1)
e2 = per_query_error_kron(Asf1[1], Wsf1)

def foo(eps):
    v = eps**2/e1 + (1-eps)**2/e2
    return np.sum(1.0 / v)

ans = optimize.minimize(foo, 0.5, method='L-BFGS-B', jac=grad(foo), bounds=[(0,1)])

print np.sqrt(baseline / ans.fun)


1.9672377749670507
2.6670582930107343


In [75]:
"""
Motivating example for union-kron parameterization:
W = (P x T) + (T x P) - i.e., 1D range queries on each dimension of a 2D domain

A kronecker product strategy is not well-suited to this workload, but a union of two products is
"""


n = 128
p = 8

I = np.eye
T = lambda n: np.ones((1,n))
P = lambda n: np.tril(np.ones((n,n)))
def pid(p, n, params):
    B = np.reshape(params, (p,n))
    A = np.vstack([I(n), B])
    return A / np.sum(A, axis=0)

wgt = 5.0
W = [[P(n),wgt*T(n)], [T(n),P(n)]]

def custom_param(params):
    S1 = pid(p, n, params[:p*n])
    S2 = pid(p, n, params[p*n:-2])
    wgt = params[-2:]
    T1 = pid(1, n, 100*np.ones(n))
    # return [[S1, S2]]
    return [[S1, T1], [T1, S2]], wgt / np.sum(wgt)

def obj(params):
    A, wgt = custom_param(params)
    ans = total_error_union_kron(A, W, wgt)
    print ans, wgt
    return ans
    
init = np.random.rand(2*p*n+2)

print obj(init)
print grad(obj)(init)
bnds = [(0,None)]*len(init)
opts={}#{'maxiter':25}

# objective = 48424 for L=1
# objective = 20713 for L=2

# without budget allocation: 270688
# with    budget allocation: 157635
res = optimize.minimize(obj, init, method='L-BFGS-B', jac=grad(obj), bounds=bnds, options=opts)

 14506651.846816534 [0.35794005 0.64205995]
14506651.846816534
Autograd ArrayBox with value 14506651.846816534 Autograd ArrayBox with value [0.35794005 0.64205995]
[ 7.22504308e+04  9.51057325e+04  8.25399329e+04 ...  1.91667923e+03
 -3.57346300e+07  1.99215903e+07]
14506651.846816534 [0.35794005 0.64205995]
Autograd ArrayBox with value 14506651.846816534 Autograd ArrayBox with value [0.35794005 0.64205995]
5234580.098260641 [0.62231096 0.37768904]
Autograd ArrayBox with value 5234580.098260641 Autograd ArrayBox with value [0.62231096 0.37768904]
5004162.8917404255 [0.64117243 0.35882757]
Autograd ArrayBox with value 5004162.8917404255 Autograd ArrayBox with value [0.64117243 0.35882757]
4752342.911022183 [0.8161707 0.1838293]
Autograd ArrayBox with value 4752342.911022183 Autograd ArrayBox with value [0.8161707 0.1838293]
4509740.217807481 [0.69695179 0.30304821]
Autograd ArrayBox with value 4509740.217807481 Autograd ArrayBox with value [0.69695179 0.30304821]
4543893.907026197 [0.80

159553.5419491788 [0.74529832 0.25470168]
Autograd ArrayBox with value 159553.5419491788 Autograd ArrayBox with value [0.74529832 0.25470168]
159487.4104740926 [0.74544197 0.25455803]
Autograd ArrayBox with value 159487.4104740926 Autograd ArrayBox with value [0.74544197 0.25455803]
159416.36270099203 [0.74551255 0.25448745]
Autograd ArrayBox with value 159416.36270099203 Autograd ArrayBox with value [0.74551255 0.25448745]
159356.14346534817 [0.74551288 0.25448712]
Autograd ArrayBox with value 159356.14346534817 Autograd ArrayBox with value [0.74551288 0.25448712]
159310.83925353063 [0.7455608 0.2544392]
Autograd ArrayBox with value 159310.83925353063 Autograd ArrayBox with value [0.7455608 0.2544392]
159247.2850828268 [0.74562002 0.25437998]
Autograd ArrayBox with value 159247.2850828268 Autograd ArrayBox with value [0.74562002 0.25437998]
159196.62894155295 [0.74561488 0.25438512]
Autograd ArrayBox with value 159196.62894155295 Autograd ArrayBox with value [0.74561488 0.25438512]
15

Autograd ArrayBox with value 158195.9770016758 Autograd ArrayBox with value [0.74560793 0.25439207]
158187.12289169084 [0.74560715 0.25439285]
Autograd ArrayBox with value 158187.12289169084 Autograd ArrayBox with value [0.74560715 0.25439285]
158176.2663377011 [0.74560432 0.25439568]
Autograd ArrayBox with value 158176.2663377011 Autograd ArrayBox with value [0.74560432 0.25439568]
158165.5977321103 [0.7455977 0.2544023]
Autograd ArrayBox with value 158165.5977321103 Autograd ArrayBox with value [0.7455977 0.2544023]
158156.23543730332 [0.74559126 0.25440874]
Autograd ArrayBox with value 158156.23543730332 Autograd ArrayBox with value [0.74559126 0.25440874]
158147.51790403642 [0.7455849 0.2544151]
Autograd ArrayBox with value 158147.51790403642 Autograd ArrayBox with value [0.7455849 0.2544151]
158138.75457619998 [0.74558774 0.25441226]
Autograd ArrayBox with value 158138.75457619998 Autograd ArrayBox with value [0.74558774 0.25441226]
158130.25575937663 [0.74559281 0.25440719]
Autog

157925.7974834417 [0.74552111 0.25447889]
Autograd ArrayBox with value 157925.7974834417 Autograd ArrayBox with value [0.74552111 0.25447889]
157924.62418679902 [0.74552142 0.25447858]
Autograd ArrayBox with value 157924.62418679902 Autograd ArrayBox with value [0.74552142 0.25447858]
157923.4535330793 [0.74552155 0.25447845]
Autograd ArrayBox with value 157923.4535330793 Autograd ArrayBox with value [0.74552155 0.25447845]
157921.74743142183 [0.74552053 0.25447947]
Autograd ArrayBox with value 157921.74743142183 Autograd ArrayBox with value [0.74552053 0.25447947]
157920.52997128607 [0.74552047 0.25447953]
Autograd ArrayBox with value 157920.52997128607 Autograd ArrayBox with value [0.74552047 0.25447953]
157919.43315144925 [0.74552037 0.25447963]
Autograd ArrayBox with value 157919.43315144925 Autograd ArrayBox with value [0.74552037 0.25447963]
157918.28819637268 [0.74551965 0.25448035]
Autograd ArrayBox with value 157918.28819637268 Autograd ArrayBox with value [0.74551965 0.254480

157857.99483569522 [0.74553348 0.25446652]
Autograd ArrayBox with value 157857.99483569522 Autograd ArrayBox with value [0.74553348 0.25446652]
157854.3665552023 [0.74552781 0.25447219]
Autograd ArrayBox with value 157854.3665552023 Autograd ArrayBox with value [0.74552781 0.25447219]
157849.6999426749 [0.74552912 0.25447088]
Autograd ArrayBox with value 157849.6999426749 Autograd ArrayBox with value [0.74552912 0.25447088]
157846.55104321433 [0.74553159 0.25446841]
Autograd ArrayBox with value 157846.55104321433 Autograd ArrayBox with value [0.74553159 0.25446841]
157842.96191022894 [0.74552734 0.25447266]
Autograd ArrayBox with value 157842.96191022894 Autograd ArrayBox with value [0.74552734 0.25447266]
157838.7861370432 [0.74553219 0.25446781]
Autograd ArrayBox with value 157838.7861370432 Autograd ArrayBox with value [0.74553219 0.25446781]
157836.95841731466 [0.74553844 0.25446156]
Autograd ArrayBox with value 157836.95841731466 Autograd ArrayBox with value [0.74553844 0.25446156

...

157637.15297323908 [0.74567234 0.25432766]
Autograd ArrayBox with value 157637.15297323908 Autograd ArrayBox with value [0.74567234 0.25432766]
157637.10954343935 [0.7456724 0.2543276]
Autograd ArrayBox with value 157637.10954343935 Autograd ArrayBox with value [0.7456724 0.2543276]
157637.05587547342 [0.7456725 0.2543275]
Autograd ArrayBox with value 157637.05587547342 Autograd ArrayBox with value [0.7456725 0.2543275]
157636.99584932034 [0.74567248 0.25432752]
Autograd ArrayBox with value 157636.99584932034 Autograd ArrayBox with value [0.74567248 0.25432752]
157636.9575727206 [0.74567249 0.25432751]
Autograd ArrayBox with value 157636.9575727206 Autograd ArrayBox with value [0.74567249 0.25432751]
157636.92766806734 [0.74567248 0.25432752]
Autograd ArrayBox with value 157636.92766806734 Autograd ArrayBox with value [0.74567248 0.25432752]
157636.89441718464 [0.7456725 0.2543275]
Autograd ArrayBox with value 157636.89441718464 Autograd ArrayBox with value [0.7456725 0.2543275]
157636

157633.73089386785 [0.74567289 0.25432711]
Autograd ArrayBox with value 157633.73089386785 Autograd ArrayBox with value [0.74567289 0.25432711]
157633.7055974963 [0.74567285 0.25432715]
Autograd ArrayBox with value 157633.7055974963 Autograd ArrayBox with value [0.74567285 0.25432715]
157633.6736252248 [0.74567282 0.25432718]
Autograd ArrayBox with value 157633.6736252248 Autograd ArrayBox with value [0.74567282 0.25432718]
157633.6487035103 [0.74567283 0.25432717]
Autograd ArrayBox with value 157633.6487035103 Autograd ArrayBox with value [0.74567283 0.25432717]
157633.61574849125 [0.74567284 0.25432716]
Autograd ArrayBox with value 157633.61574849125 Autograd ArrayBox with value [0.74567284 0.25432716]
157633.58339302937 [0.74567283 0.25432717]
Autograd ArrayBox with value 157633.58339302937 Autograd ArrayBox with value [0.74567283 0.25432717]
157633.5473418851 [0.74567283 0.25432717]
Autograd ArrayBox with value 157633.5473418851 Autograd ArrayBox with value [0.74567283 0.25432717]


Autograd ArrayBox with value 157632.85129060203 Autograd ArrayBox with value [0.74567316 0.25432684]
157632.84494248015 [0.74567315 0.25432685]
Autograd ArrayBox with value 157632.84494248015 Autograd ArrayBox with value [0.74567315 0.25432685]
157632.83842187468 [0.74567313 0.25432687]
Autograd ArrayBox with value 157632.83842187468 Autograd ArrayBox with value [0.74567313 0.25432687]
157632.83249988122 [0.74567312 0.25432688]
Autograd ArrayBox with value 157632.83249988122 Autograd ArrayBox with value [0.74567312 0.25432688]
157632.82796964044 [0.74567311 0.25432689]
Autograd ArrayBox with value 157632.82796964044 Autograd ArrayBox with value [0.74567311 0.25432689]
157632.824629403 [0.74567309 0.25432691]
Autograd ArrayBox with value 157632.824629403 Autograd ArrayBox with value [0.74567309 0.25432691]
157632.818981839 [0.74567308 0.25432692]
Autograd ArrayBox with value 157632.818981839 Autograd ArrayBox with value [0.74567308 0.25432692]
157632.81425255476 [0.74567304 0.25432696]


157632.5603084757 [0.74567328 0.25432672]
Autograd ArrayBox with value 157632.5603084757 Autograd ArrayBox with value [0.74567328 0.25432672]
157632.55725390743 [0.74567328 0.25432672]
Autograd ArrayBox with value 157632.55725390743 Autograd ArrayBox with value [0.74567328 0.25432672]
157632.55378533865 [0.74567329 0.25432671]
Autograd ArrayBox with value 157632.55378533865 Autograd ArrayBox with value [0.74567329 0.25432671]


KeyboardInterrupt: 

In [89]:
Asf1[0][2][70]

array([0.45013677, 0.03134619, 0.05427473, 0.45583274, 0.05427473,
       0.45584835, 0.45586493, 0.45741498, 0.05427473, 0.45585822,
       0.45588767, 0.45745975, 0.45591327, 0.45735957, 0.45737857,
       0.45565821, 0.05427473, 0.45581349, 0.45584342, 0.45737104,
       0.45587148, 0.4575037 , 0.45736559, 0.45565342, 0.45585948,
       0.45741837, 0.4573553 , 0.45565155, 0.45737033, 0.45566188,
       0.45565914, 0.45268668, 0.05427473, 0.45585213, 0.45585106,
       0.45737325, 0.45583751, 0.45738388, 0.45740147, 0.45565477,
       0.45595816, 0.45740834, 0.45755528, 0.45566344, 0.45745386,
       0.45565316, 0.45566807, 0.45268732, 0.455844  , 0.4573552 ,
       0.45728871, 0.45565699, 0.45737627, 0.45566591, 0.45566149,
       0.45268667, 0.45739529, 0.45565973, 0.4556571 , 0.45268537,
       0.45565313, 0.45268692, 0.45268392, 0.45106328])

In [101]:
W = P(8)
A = T(8)
AtA = np.dot(A.T, A)
AtA1 = np.linalg.pinv(AtA)
AA1 = np.dot(AtA1, AtA)
X = np.dot(W, AA1) - W
np.where(X.any(axis=1), np.inf, 0)

array([inf, inf, inf, inf, inf, inf, inf,  0.])

In [110]:
~X.any(axis=1)

array([False, False, False, False, False, False, False,  True])